In [1]:
from utils.config import *
from agents.tester import Tester
from utils.metrics import *

import torchaudio
torchaudio.set_audio_backend("soundfile")


# JSON = 'configs/pre_trained/ResNetSE34L.json'
# JSON = 'configs/pre_trained/ResNetSE34V2.json'
# JSON = 'configs/pre_trained/AutoSpeech.json'
# JSON = 'configs/pre_trained/ResNet18.json'
# JSON = 'configs/pre_trained/ResNet34.json'
# JSON = 'configs/pre_trained/Cnn3D.json'
# JSON = 'configs/pre_trained/SincNet.json'
# JSON = 'configs/pre_trained/MobileNet.json'

config = process_config(JSON)
tester = Tester(config)

[INFO]: Experiment: MobileNet Pretrained.
[INFO]: Operation will be on ***** GPU-CUDA ***** 
[INFO]: __Python VERSION:  3.9.5 | packaged by conda-forge | (default, Jun 19 2021, 00:22:33) [MSC v.1916 64 bit (AMD64)]
[INFO]: __pyTorch VERSION:  1.9.0
[INFO]: __CUDA VERSION
[INFO]: __CUDNN VERSION:  7605
[INFO]: __Number CUDA Devices:  1
[INFO]: __Devices
[INFO]: Active CUDA Device: GPU 0
[INFO]: Available devices  1
[INFO]: Current cuda device  0
[INFO]: Loading checkpoint 'experiments\MobileNet Pretrained\checkpoints/pretrained_weights/MobileNet_128.pkl'
[INFO]: No checkpoint exists from 'experiments\MobileNet Pretrained\checkpoints/'. Skipping...


In [2]:
model = tester.__model__
input_size = next(iter(tester.loader))[0].squeeze(0).shape
input_size

torch.Size([90881])

In [3]:
from ptflops import get_model_complexity_info


def constructor(input_res):
    return {'x': torch.randn(*input_res).cuda()}


macs, params = get_model_complexity_info(model, (1, *input_size), input_constructor=constructor, as_strings=True, verbose=False, print_per_layer_stat=False)
print('{:<30}  {:<8}'.format('Computational complexity: ', macs))

C:\Users\giang\anaconda3\envs\thesis\lib\site-packages\torch\nn\functional.py:652: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool1d(input, kernel_size, stride, padding, dilation, ceil_mode)


Warning! No positional inputs found for a module, assuming batch size is 1.
Computational complexity:       4.25 GMac


In [3]:
print('Computing parameters...')
total_params, trainable_params = compute_parameters(model)
print('Computing complexity...')
avg_flops_cost = compute_complexity(model, input_size)
print('Computing mean inference time...')
mean_tfp, std_tfp = compute_inference_time(model, input_size)
print('Computing memory usage...')
memory = compute_memory_usage(model, input_size, model_memory=trainer.required_memory)

Computing parameters...
Computing complexity...


C:\Users\giang\anaconda3\envs\thesis\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Computing mean inference time...
Computing memory usage...


In [4]:
print('================================================================')
print(str(model.__class__).split('.')[-1].split("'")[0])
print('================================================================')
print('Total params: ' + str(total_params))
print('Trainable params: ' + str(trainable_params))
print('Non-trainable params: ' + str(total_params - trainable_params))
print('----------------------------------------------------------------')
print(f'Average flops cost: {avg_flops_cost}')
print('----------------------------------------------------------------')
print('Average inference time for every batch size:')
batch_sizes = [1, 2, 4, 8, 16, 32, 64]
for b, m, s in zip(batch_sizes, mean_tfp, std_tfp):
    print(f'{b}: {m:.2f} +/- {s:.2f}')
print('----------------------------------------------------------------')
print('Memory usage for every batch size:')
batch_sizes = [1, 2, 4, 8, 16, 32, 64]
for b, m in zip(batch_sizes, memory):
    print(f'{b}: {m}')
print('================================================================')

AutoSpeechResNet34
Total params: 21920163
Trainable params: 21920163
Non-trainable params: 0
----------------------------------------------------------------
Average flops cost: 5.59681536
----------------------------------------------------------------
Average inference time for every batch size:
1: 12.19 +/- 0.87
2: 8.54 +/- 0.35
4: 6.30 +/- 0.15
8: 5.61 +/- 0.04
16: 4.61 +/- 0.05
32: 4.29 +/- 0.02
64: 4.97 +/- 0.01
----------------------------------------------------------------
Memory usage for every batch size:
1: 546.0
2: 546.0
4: 546.0
8: 546.0
16: 546.0
32: 546.0
64: 546.0
